# VacationPy
----
#### Author: Amir Vazquez
Date: 12/31/2020
#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
try:
    import matplotlib.pyplot as plt
    from scipy.stats import linregress
    import pandas as pd
    import numpy as np
    import requests
    import gmaps
    import os, sys
    from datetime import datetime

    # Import API key
    sys.path.append('..\\apikeys')
    from api_keys import g_key
    
    #from my_func import lregression
    
    # Configure gmaps
    gmaps.configure(api_key=g_key)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
except Exception as e:
    print(f"a module(s) have not been imported {e}" )

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../starter_code/output_data/cities.csv")
city_weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa maria,-29.68,-53.81,18.60,82,72,2.73,BR,1609746832
1,1,dikson,73.51,80.55,-20.28,86,80,7.79,RU,1609746832
2,2,neepawa,50.23,-99.47,1.00,86,90,10.80,CA,1609746832
3,3,san quintin,30.48,-115.95,13.07,88,0,2.10,MX,1609746833
4,4,cherskiy,68.75,161.30,-33.24,85,99,2.17,RU,1609746833


In [3]:
# update Date from unixtime to date
city_weather_df['Date'] = pd.to_datetime(city_weather_df['Date'], unit='s')

city_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa maria,-29.68,-53.81,18.60,82,72,2.73,BR,2021-01-04 07:53:52
1,1,dikson,73.51,80.55,-20.28,86,80,7.79,RU,2021-01-04 07:53:52
2,2,neepawa,50.23,-99.47,1.00,86,90,10.80,CA,2021-01-04 07:53:52
3,3,san quintin,30.48,-115.95,13.07,88,0,2.10,MX,2021-01-04 07:53:53
4,4,cherskiy,68.75,161.30,-33.24,85,99,2.17,RU,2021-01-04 07:53:53


In [4]:
#Drop column Unnamed:0
city_weather_df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
city_weather_df.dtypes 

City                  object
Lat                  float64
Lng                  float64
Max Temp             float64
Humidity               int64
Cloudiness             int64
Wind Speed           float64
Country               object
Date          datetime64[ns]
dtype: object

In [6]:
city_weather_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
count,564.000000,564.000000,564.00000,564.000000,564.000000,564.000000
mean,21.048954,18.682571,9.78117,74.381206,59.051418,3.514255
std,33.362696,91.386952,17.73852,19.864507,39.380990,2.593409
min,-54.800000,-179.170000,-45.97000,1.000000,0.000000,0.050000
25%,-5.335000,-64.875000,0.42000,67.000000,20.000000,1.515000
50%,28.020000,23.140000,13.95500,80.000000,75.000000,2.740000
75%,48.162500,96.860000,25.00000,88.000000,95.000000,4.600000
max,78.220000,179.320000,38.68000,100.000000,100.000000,13.400000


### Humidity Heatmap
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# store values of location (latitude and longitude) into loc[] for use
loc = city_weather_df[["Lat","Lng"]]
# store values of humidity into hum[] for use
hum = city_weather_df["Humidity"].astype(float)


In [8]:
# Heatmap plot 
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
fig1 = gmaps.figure()
new_york_coordinates = (40.75, -74.00)

heat_layer = gmaps.heatmap_layer(loc, 
                                 weights = hum, 
                                 dissipating = False, 
                                 max_intensity = 200,
                                 point_radius = 3)

fig1 = gmaps.figure(map_type='TERRAIN' , center=new_york_coordinates, zoom_level=2,
                    layout={'width':'1000px', 
                            'height': '500px',
                            'border': '1px solid black', 
                            'padding': '1px' 
                    
                           })

fig1.add_layer(heat_layer)

fig1
#fig1.save("output_data/worldheatmap.png")

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [9]:
# Remove rows missing data
city_df = city_weather_df

maxtempf = city_df['Max Temp']
windspeedmph = city_df['Wind Speed']

# Convert temperature from C to F and add a new column
city_df['Max Temp F'] = maxtempf*(9/5)+32
# Convert Wind Speed from Km to MPH and add a new column
city_df['Wind Speed (mph)'] = windspeedmph/1.609
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph)
0,santa maria,-29.68,-53.81,18.60,82,72,2.73,BR,2021-01-04 07:53:52,65.480,1.696706
1,dikson,73.51,80.55,-20.28,86,80,7.79,RU,2021-01-04 07:53:52,-4.504,4.841516
2,neepawa,50.23,-99.47,1.00,86,90,10.80,CA,2021-01-04 07:53:52,33.800,6.712244
3,san quintin,30.48,-115.95,13.07,88,0,2.10,MX,2021-01-04 07:53:53,55.526,1.305158
4,cherskiy,68.75,161.30,-33.24,85,99,2.17,RU,2021-01-04 07:53:53,-27.832,1.348664


In [31]:
# this combination will only pull 9 records
city_df_upd = city_df.loc[(city_df['Max Temp F'] > 70) & (city_df['Max Temp F'] < 80)]
city_df_upd = city_df_upd.loc[city_df_upd['Wind Speed (mph)']< 4 ]
city_df_upd = city_df_upd.loc[city_df_upd['Cloudiness'] == 0]

# this combination will bring 35 potential vacation spots its ok if its a little hotter
# city_df_upd = city_df.loc[(city_df['Max Temp F']>70) & (city_df['Max Temp F']< 90)]
# city_df_upd = city_df_upd.loc[city_df_upd['Wind Speed (mph)']< 10 ]
# city_df_upd = city_df_upd.loc[city_df_upd['Cloudiness']<2]
                          
city_df_upd.dropna()
city_df_upd.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Max Temp F,Wind Speed (mph)
count,14.000000,14.000000,14.000000,14.000000,14.0,14.000000,14.000000,14.000000
mean,3.199286,1.747857,24.135714,50.142857,0.0,3.179286,75.444286,1.975939
std,22.330594,62.211955,1.272355,26.150032,0.0,1.563416,2.290239,0.971669
min,-33.930000,-134.970000,22.000000,18.000000,0.0,1.390000,71.600000,0.863891
25%,-16.422500,-13.532500,23.165000,19.250000,0.0,1.927500,73.697000,1.197949
50%,12.740000,9.675000,24.260000,57.500000,0.0,2.700000,75.668000,1.678061
75%,18.237500,28.525000,25.000000,66.250000,0.0,4.192500,77.000000,2.605656
max,27.980000,94.670000,26.240000,86.000000,0.0,6.390000,79.232000,3.971411


In [32]:
city_df_upd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph)
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,2021-01-04 07:53:55,79.232,1.162213
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,2021-01-04 07:53:59,77.666,3.971411
87,cape town,-33.93,18.42,25.00,61,0,1.50,ZA,2021-01-04 07:50:19,77.000,0.932256
180,keti bandar,24.14,67.45,23.53,19,0,4.35,PK,2021-01-04 07:54:35,74.354,2.703543
195,saldanha,-33.01,17.94,22.00,68,0,5.70,ZA,2021-01-04 07:54:38,71.600,3.542573


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# Hotel variables
hotel_df = city_df_upd.copy()
hotel_df['Hotel Name']=''
hotel_df['Vicinity']=''
hotel_df['Rating']=''
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph),Hotel Name,Vicinity,Rating
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,2021-01-04 07:53:55,79.232,1.162213,,,
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,2021-01-04 07:53:59,77.666,3.971411,,,
87,cape town,-33.93,18.42,25.00,61,0,1.50,ZA,2021-01-04 07:50:19,77.000,0.932256,,,
180,keti bandar,24.14,67.45,23.53,19,0,4.35,PK,2021-01-04 07:54:35,74.354,2.703543,,,
195,saldanha,-33.01,17.94,22.00,68,0,5.70,ZA,2021-01-04 07:54:38,71.600,3.542573,,,


In [35]:
# geocoordinates
target_coordinates = "0, 0"
target_search = "hotel"
target_radius = 10000
#target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
 #   "type": target_type,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    #restr_type = row['ethnicity']
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    cityname = row["City"]
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {target_search} in {cityname} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Vicinity']   = results[0]['vicinity']
        #Add Hotel Rating to the df
        hotel_df.loc[index, 'Rating'] = results[0]['rating']
# for testing        
#        print(response["results"][0]["name"])
#        print(response["results"][0]["vicinity"])
#        print(response["results"][0]["rating"])
        
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping.")
        
    print("------------")
print("Search End ----------------------")    

Closest hotel in san-pedro is Afrikcasa.
------------
Closest hotel in rikitea is Pension Maro'i.
------------
Closest hotel in cape town is The Silo Hotel.
------------
Missing field/result... skipping.
------------
Closest hotel in saldanha is Hoedjiesbaai Hotel.
------------
Closest hotel in zihuatanejo is Hotel Villas Punta Blanca.
------------
Closest hotel in kwekwe is EAGLESVIEW GUEST LODGE/HOTEL.
------------
Closest hotel in sao jose da coroa grande is Perobavilla Hotel.
------------
Missing field/result... skipping.
------------
Closest hotel in bubaque is Eco Lodge.
------------
Closest hotel in shahdadkot is Ali Hassan Naich Hotel.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Search End ----------------------


In [37]:
hotel_df.head(30)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph),Hotel Name,Vicinity,Rating
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,2021-01-04 07:53:55,79.232,1.162213,Afrikcasa,"Pédro, San",4
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,2021-01-04 07:53:59,77.666,3.971411,Pension Maro'i,Gambier,4
87,cape town,-33.93,18.42,25.00,61,0,1.50,ZA,2021-01-04 07:50:19,77.000,0.932256,The Silo Hotel,"Silo Square, Victoria & Alfred Waterfront, Cap...",4.7
180,keti bandar,24.14,67.45,23.53,19,0,4.35,PK,2021-01-04 07:54:35,74.354,2.703543,,,
195,saldanha,-33.01,17.94,22.00,68,0,5.70,ZA,2021-01-04 07:54:38,71.600,3.542573,Hoedjiesbaai Hotel,"38 Main Rd, Saldanha",3.8
284,zihuatanejo,17.63,-101.55,24.10,60,0,1.79,MX,2021-01-04 07:55:00,75.380,1.112492,Hotel Villas Punta Blanca,"Avenida Lote #128/129 Playa Blanca, Zihuatanejo",4.2
302,kwekwe,-18.93,29.81,24.42,58,0,2.56,ZW,2021-01-04 07:55:04,75.956,1.591050,EAGLESVIEW GUEST LODGE/HOTEL,"Unnamed Road, Kwekwe",4.3
319,sao jose da coroa grande,-8.90,-35.15,24.72,86,0,4.49,BR,2021-01-04 07:55:08,76.496,2.790553,Perobavilla Hotel,"PRAIA DE PEROBA, AL-101, 10, Maragogi",4.3
321,goundam,16.41,-3.67,22.33,19,0,3.26,ML,2021-01-04 07:55:08,72.194,2.026103,,,
383,bubaque,11.28,-15.83,25.62,52,0,3.72,GW,2021-01-04 07:55:23,78.116,2.311995,Eco Lodge,"Angurman,",5


In [38]:
# 3 records with NAN will be dropping these values
hotel_df= hotel_df.dropna()
#Some records do not come back with NaN well search for blank
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] !=''] 
hotel_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Max Temp F,Wind Speed (mph)
count,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,-6.264444,-17.784444,24.504444,63.111111,0.0,3.267778,76.108000,2.030937
std,22.627542,64.589541,1.313317,20.949012,0.0,1.892484,2.363971,1.176186
min,-33.930000,-134.970000,22.000000,18.000000,0.0,1.390000,71.600000,0.863891
25%,-23.120000,-35.150000,24.100000,58.000000,0.0,1.790000,75.380000,1.112492
50%,-8.900000,-6.640000,24.720000,61.000000,0.0,2.560000,76.496000,1.591050
75%,11.280000,18.420000,25.370000,80.000000,0.0,4.490000,77.666000,2.790553
max,27.850000,67.910000,26.240000,86.000000,0.0,6.390000,79.232000,3.971411


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, 
                                 info_box_content=hotel_info)
# Display figure
#Heatmap plot 
fig1.add_layer(hotel_layer)

location_marker = gmaps.marker_layer(locations)
#fig1.add_layer(heat_layer)
fig1.add_layer(location_marker)

fig1


Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='1000px'))

### Analysis
In review with the possible vacation spots that are 1 great in weather. I decited to venture into Mexico into Zihuatanejo "Hotel Villas Punta Blanca". 2nd option is Brazil Sao Jose de Coroa Grande, and our 3rd option is The Silo Hotel in Cape Town.

I did added a rating of the hotel to help with the decision on which place to stay.
